## Putting together a naive dataset to create a FSM

In [1]:
# test file collect:
import pandas as pd

# real property sales df create:
rps = pd.read_csv("../data/EXTR_RPSale.csv")

/opt/anaconda3/envs/oy-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (1,2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [2]:
# real property sales df create:
parcel = pd.read_csv("../data/EXTR_Parcel.csv", encoding = 'latin-1')

# real property sales df create:
res_build = pd.read_csv("../data/EXTR_ResBldg.csv")

/opt/anaconda3/envs/oy-env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
# pad major/minor
def major_pad(row):
    """
    This function takes an index item and pads it with 0's at the beginning
    to make it's total length equal to the pad_to value. 
    
    arguments:
    row (str):  item to add 0's infront of
    pad_to (int):  total length of padded item
    """
    num_zeros = 6 - len(row)
    return num_zeros * '0' + row

def minor_pad(row):
    """
    This function takes an index item and pads it with 0's at the beginning
    to make it's total length equal to the pad_to value. 
    
    arguments:
    row (str):  item to add 0's infront of
    pad_to (int):  total length of padded item
    """
    num_zeros = 4 - len(row)
    return num_zeros * '0' + row

In [4]:
## turn this into a function 

tables = [rps, parcel, res_build]

for table in tables:
    # change minor/minor to str
    table['Major'] = table['Major'].astype(str)
    table['Minor'] = table['Minor'].astype(str)
    
    # add padding to major/minor
    table['Major'] = table['Major'].apply(major_pad)
    table['Minor'] = table['Minor'].apply(minor_pad)
    
    # add maj_min column
    table['maj_min'] = table['Major'] + table['Minor']
    
    # drop old major/minor columns
    table.drop(labels = ['Major', 'Minor'], axis = 1, inplace = True)

In [5]:
# I would have done this in the above for loop but I already ran it and didn't want to run again
for table in tables:
    table.set_index(keys = 'maj_min', inplace = True)

In [6]:
data = rps.merge(res_build, on = 'maj_min')

In [7]:
data = data.merge(parcel, on = 'maj_min')

In [8]:
data.head()

,ExciseTaxNbr,DocumentDate,SalePrice,RecordingNbr,Volume,Page,PlatNbr,PlatType,PlatLot_x,PlatBlock_x,...,SeismicHazard,LandslideHazard,SteepSlopeHazard,Stream,Wetland,SpeciesOfConcern,SensitiveAreaTract,WaterProblems,TranspConcurrency,OtherProblems
maj_min,,,,,,,,,,,,,,,,,,,,,
1388600110,2687551,08/21/2014,245000,20140828001436,,,,,,,...,N,N,N,N,N,N,N,N,N,N
1388600110,1066933,06/12/1989,109300,198906191556,144,021,138860,P,11,,...,N,N,N,N,N,N,N,N,N,N
1388600110,2102034,01/16/2005,14684,20050216001248,,,,,,,...,N,N,N,N,N,N,N,N,N,N
1388600110,2131568,06/08/2005,0,20050617000041,,,,,,,...,N,N,N,N,N,N,N,N,N,N
4239430050,2704079,10/11/2014,0,20141205000558,,,,,,,...,N,N,N,N,N,N,N,N,N,N


We can see from this df's head that we have multiple entries with the same index...

In [14]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1492777 entries, 1388600110 to 3586500040
Columns: 150 entries, ExciseTaxNbr to year
dtypes: bool(1), datetime64[ns](1), float64(5), int64(84), object(59)
memory usage: 1.7+ GB


This is obviously way too big and we can filter down straight away by year... 

In [10]:
data['DocumentDate'] = pd.to_datetime(data['DocumentDate'])

In [11]:
data['year'] = data['DocumentDate'].dt.year

In [12]:
data2019 = data[data['year'] == 2019]

In [13]:
data2019.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43867 entries, 2130430120 to 0869650030
Columns: 150 entries, ExciseTaxNbr to year
dtypes: bool(1), datetime64[ns](1), float64(5), int64(84), object(59)
memory usage: 50.2+ MB


## Create FSM:

In [15]:
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [19]:
# create x, y variables to find potential correlations:

X = data2019
y = data2019['SalePrice']

In [26]:
# create correlation matrix:
corr_x = X.corr()


In [25]:
corr_x['SalePrice'].sort_values(ascending = False)

SalePrice        1.000000
BldgGrade        0.203755
SqFtTotLiving    0.195816
SqFt2ndFloor     0.157990
BathFullCount    0.132185
                   ...   
SpecArea              NaN
SpecSubArea           NaN
Unbuildable           NaN
AirportNoise          NaN
year                  NaN
Name: SalePrice, Length: 90, dtype: float64

In [ ]:
# set up model:
x_val = str(BldgGrade)
f = 'SalePrice ~'+ x_val
model = ols(formula = f, data = data).fit()

Luluva:
property type 
date
sale price > 0 
principal use - residential and condiminum 
property class - remove land
18,000 entries